## Config

In [48]:
import pandas as pd 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format
import numpy as np 
from datetime import timedelta

## Data

### Train

In [49]:
df_train = pd.read_csv('train.csv')
df_train['date'] = pd.to_datetime(df_train['date'])
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0
1,1,2013-01-01,1,BABY CARE,0.00,0
2,2,2013-01-01,1,BEAUTY,0.00,0
3,3,2013-01-01,1,BEVERAGES,0.00,0
4,4,2013-01-01,1,BOOKS,0.00,0


### Store info

In [50]:
df_stores = pd.read_csv('stores.csv')
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Calendar information

In [51]:
df_holiday = pd.read_csv('holidays_events.csv')
df_holiday['date'] = pd.to_datetime(df_holiday['date'])
df_holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         350 non-null    datetime64[ns]
 1   type         350 non-null    object        
 2   locale       350 non-null    object        
 3   locale_name  350 non-null    object        
 4   description  350 non-null    object        
 5   transferred  350 non-null    bool          
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 14.1+ KB


In [52]:
# ['Holiday' nan 'Work Day' 'Additional' 'Event' 'Transfer' 'Bridge']
# Mark transferred holidays as normal days and find the correct celebration dates
for index, row in df_holiday[df_holiday['transferred']].iterrows():
    # Find the new date for this transferred holiday
    transfer_row = df_holiday[(df_holiday['description'] == row['description']) & (df_holiday['type'] == 'Transfer')]
    if not transfer_row.empty:
        # Assuming only one row matches, otherwise adjust as necessary
        new_date_index = transfer_row.index[0]

df_holiday['bridge'] = False
df_holiday['holiday_national'] = False
df_holiday['holiday_local'] = False
df_holiday['holiday_regional'] = False 

df_holiday.loc[df_holiday['type'] == 'Bridge', 'bridge'] = True
df_holiday.loc[df_holiday['locale'] == 'National', 'holiday_national'] = True  # Mark regular holidays
df_holiday.loc[df_holiday['type'] == 'Local', 'holiday_local'] = True  # Mark bridge days as holidays
df_holiday.loc[df_holiday['type'] == 'Regional', 'holiday_regional'] = True

In [53]:
df_holiday['transferred'] = df_holiday['transferred'].astype(int)
df_holiday['holiday_national'] = df_holiday['holiday_national'].astype(int)
df_holiday['holiday_regional'] = df_holiday['holiday_regional'].astype(int)
df_holiday['holiday_local'] = df_holiday['holiday_local'].astype(int)
df_holiday['bridge'] = df_holiday['bridge'].astype(int)
df_holiday['type'] = df_holiday['type'].astype('category')
df_holiday['locale'] = df_holiday['locale'].astype('category')
# df_holiday['work_day'] = df_holiday['work_day'].astype(int)
# df_holiday['event'] = df_holiday['event'].astype(int)

df_holiday.head()

,date,type,locale,locale_name,description,transferred,bridge,holiday_national,holiday_local,holiday_regional
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,0,0,0,0,0
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,0,0,0,0,0
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,0,0,0,0,0
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,0,0,0,0,0
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,0,0,0,0,0


In [54]:
df_holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              350 non-null    datetime64[ns]
 1   type              350 non-null    category      
 2   locale            350 non-null    category      
 3   locale_name       350 non-null    object        
 4   description       350 non-null    object        
 5   transferred       350 non-null    int64         
 6   bridge            350 non-null    int64         
 7   holiday_national  350 non-null    int64         
 8   holiday_local     350 non-null    int64         
 9   holiday_regional  350 non-null    int64         
dtypes: category(2), datetime64[ns](1), int64(5), object(2)
memory usage: 23.0+ KB


### Oil prices

In [55]:
df_oil = pd.read_csv('oil.csv')
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [56]:
df_oil.loc[df_oil['dcoilwtico'] == 0]
# No values are zero

,date,dcoilwtico


In [57]:
n_obs_oil = len(df_oil['dcoilwtico'])
print(f'Number of obs oil: {n_obs_oil}')

n_obs_na = df_oil['dcoilwtico'].isna().sum()

print(f'Number of obs isna: {n_obs_na}')
print(f'Proportion missing: {n_obs_na/n_obs_oil}')

Number of obs oil: 1218
Number of obs isna: 43
Proportion missing: 0.035303776683087026


#### Simple interpolation

In [58]:
df_oil = df_oil.set_index(df_oil['date'])
df_oil = df_oil['dcoilwtico'].interpolate(method = 'time')

In [59]:
# Set first manually, just set to the next day 93.14
df_oil.iloc[0] = 93.14

In [60]:
df_oil.isna().sum()

0

In [61]:
df_oil.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 1218 entries, 2013-01-01 to 2017-08-31
Series name: dcoilwtico
Non-Null Count  Dtype  
--------------  -----  
1218 non-null   float64
dtypes: float64(1)
memory usage: 19.0 KB


### Transactions

In [62]:
df_transactions = pd.read_csv('transactions.csv')
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
df_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


## Merge dataframes

In [63]:
df_complete = pd.merge(df_train, df_oil, on = 'date', how = 'left')
df_complete.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,93.14
1,1,2013-01-01,1,BABY CARE,0.00,0,93.14
2,2,2013-01-01,1,BEAUTY,0.00,0,93.14
3,3,2013-01-01,1,BEVERAGES,0.00,0,93.14
4,4,2013-01-01,1,BOOKS,0.00,0,93.14


In [64]:
df_complete = pd.merge(df_complete, df_holiday, on = 'date', how = 'left')
df_complete.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type,locale,locale_name,description,transferred,bridge,holiday_national,holiday_local,holiday_regional
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00
1,1,2013-01-01,1,BABY CARE,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00
2,2,2013-01-01,1,BEAUTY,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00
3,3,2013-01-01,1,BEVERAGES,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00
4,4,2013-01-01,1,BOOKS,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00


In [65]:
df_complete = pd.merge(df_complete, df_stores, on = 'store_nbr', how = 'left')
df_complete.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type_x,locale,locale_name,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13


In [66]:
df_complete = pd.merge(df_complete, df_transactions, on = ['store_nbr', 'date'], how = 'left')
df_complete.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type_x,locale,locale_name,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Quito,Pichincha,D,13,NaN


In [67]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 21 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                int64         
 1   date              datetime64[ns]
 2   store_nbr         int64         
 3   family            object        
 4   sales             float64       
 5   onpromotion       int64         
 6   dcoilwtico        float64       
 7   type_x            category      
 8   locale            category      
 9   locale_name       object        
 10  description       object        
 11  transferred       float64       
 12  bridge            float64       
 13  holiday_national  float64       
 14  holiday_local     float64       
 15  holiday_regional  float64       
 16  city              object        
 17  state             object        
 18  type_y            object        
 19  cluster           int64         
 20  transactions      float64       
dtypes: categ

## Processing and conversion

### Filter out entries where the stores have not opened yet

In [68]:
# Filter rows where sales are greater than zero
sales_not_zero = df_complete[df_complete['sales'] > 0]

# Group by 'store_nbr' and find the first date of non-zero sales for each store
first_non_zero_sales_date_per_store = sales_not_zero.groupby('store_nbr')['date'].min().reset_index()

# Rename columns for clarity
first_non_zero_sales_date_per_store.rename(columns={'date': 'first_sale_date'}, inplace=True)

# Display the result
first_non_zero_sales_date_per_store.head()

,store_nbr,first_sale_date
0,1,2013-01-02
1,2,2013-01-02
2,3,2013-01-02
3,4,2013-01-02
4,5,2013-01-02


In [70]:
# Merge the main DataFrame with the first sale date information
df_with_first_sale_date = pd.merge(df_complete, first_non_zero_sales_date_per_store, on='store_nbr', how='left')

# Filter rows where the store's sale date is on or after the first sale date
df_opened_stores = df_with_first_sale_date[df_with_first_sale_date['date'] >= df_with_first_sale_date['first_sale_date']]

df_opened_stores = df_opened_stores.drop(columns=['first_sale_date'])

In [72]:
df_opened_stores.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type_x,locale,locale_name,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster,transactions
561,561,2013-01-01,25,AUTOMOTIVE,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00
562,562,2013-01-01,25,BABY CARE,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00
563,563,2013-01-01,25,BEAUTY,2.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00
564,564,2013-01-01,25,BEVERAGES,810.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00
565,565,2013-01-01,25,BOOKS,0.00,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00


### Pivot dataframe to have sales amount per product family

In [75]:
# Pivot the DataFrame to get sales amounts for each family category as separate columns
sales_pivot = df_opened_stores.pivot_table(index=['date', 'store_nbr'], 
                                      columns='family', 
                                      values='sales', 
                                      fill_value=0).reset_index()

# Reset index to make the DataFrame flat
sales_pivot.reset_index(inplace=True, drop=True)

subset_df = df_opened_stores.drop(columns=['sales', 'family']).drop_duplicates(subset=['date', 'store_nbr'])
# Optionally, rename the columns if you want to add a prefix or modify them in any way
# sales_pivot.columns = ['CustomPrefix_' + str(col) for col in sales_pivot.columns]
final_df = pd.merge(subset_df, sales_pivot, on=['date', 'store_nbr'], how='left')
# Display the result
final_df.head()

,id,date,store_nbr,onpromotion,dcoilwtico,type_x,locale,locale_name,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster,transactions,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,EGGS,FROZEN FOODS,GROCERY I,GROCERY II,HARDWARE,HOME AND KITCHEN I,HOME AND KITCHEN II,HOME APPLIANCES,HOME CARE,LADIESWEAR,LAWN AND GARDEN,LINGERIE,"LIQUOR,WINE,BEER",MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,561,2013-01-01,25,0,93.14,Holiday,National,Ecuador,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00,0.00,0.00,2.00,810.00,0.00,180.59,0.00,186.00,143.00,71.09,46.00,29.65,700.00,15.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,5.00,105.00,0.00,110.80,25.00,0.00,0.00,42.64,37.85,0.00,0.00,0.00
1,1782,2013-01-02,1,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,2111.00,2.00,0.00,2.00,1091.00,0.00,470.65,0.00,1060.00,579.00,164.07,246.00,131.00,2652.00,31.00,3.00,0.00,0.00,0.00,0.00,0.00,10.00,13.00,67.00,0.00,369.10,194.00,0.00,0.00,247.30,47.00,0.00,0.00,38.03
2,1815,2013-01-02,10,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,C,15,1293.00,3.00,0.00,6.00,396.00,0.00,151.00,0.00,1110.00,101.00,276.00,57.00,41.16,2579.00,8.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,6.00,0.00,291.14,223.00,0.00,0.00,146.78,26.00,0.00,0.00,3.00
3,1848,2013-01-02,11,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cayambe,Pichincha,B,6,3547.00,12.00,0.00,7.00,3443.00,0.00,763.00,0.00,3260.00,296.00,527.29,140.00,57.77,7736.00,15.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,29.00,0.00,832.54,604.00,0.00,0.00,548.03,31.00,0.00,0.00,5.00
4,1881,2013-01-02,12,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Latacunga,Cotopaxi,C,15,1362.00,4.00,0.00,1.00,682.00,0.00,337.00,0.00,1092.00,151.00,176.00,75.00,14.00,2677.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,36.00,0.00,232.35,210.00,0.00,0.00,103.17,41.00,0.00,0.00,9.00


#### Drop unecessary columns

In [77]:
final_df.drop(columns = ['type_x', 'locale', 'locale_name'], inplace = True)
final_df.head(10)

,id,date,store_nbr,onpromotion,dcoilwtico,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster,transactions,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,EGGS,FROZEN FOODS,GROCERY I,GROCERY II,HARDWARE,HOME AND KITCHEN I,HOME AND KITCHEN II,HOME APPLIANCES,HOME CARE,LADIESWEAR,LAWN AND GARDEN,LINGERIE,"LIQUOR,WINE,BEER",MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,561,2013-01-01,25,0,93.14,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00,0.00,0.00,2.00,810.00,0.00,180.59,0.00,186.00,143.00,71.09,46.00,29.65,700.00,15.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,5.00,105.00,0.00,110.80,25.00,0.00,0.00,42.64,37.85,0.00,0.00,0.00
1,1782,2013-01-02,1,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,2111.00,2.00,0.00,2.00,1091.00,0.00,470.65,0.00,1060.00,579.00,164.07,246.00,131.00,2652.00,31.00,3.00,0.00,0.00,0.00,0.00,0.00,10.00,13.00,67.00,0.00,369.10,194.00,0.00,0.00,247.30,47.00,0.00,0.00,38.03
2,1815,2013-01-02,10,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,C,15,1293.00,3.00,0.00,6.00,396.00,0.00,151.00,0.00,1110.00,101.00,276.00,57.00,41.16,2579.00,8.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,6.00,0.00,291.14,223.00,0.00,0.00,146.78,26.00,0.00,0.00,3.00
3,1848,2013-01-02,11,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Cayambe,Pichincha,B,6,3547.00,12.00,0.00,7.00,3443.00,0.00,763.00,0.00,3260.00,296.00,527.29,140.00,57.77,7736.00,15.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,29.00,0.00,832.54,604.00,0.00,0.00,548.03,31.00,0.00,0.00,5.00
4,1881,2013-01-02,12,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Latacunga,Cotopaxi,C,15,1362.00,4.00,0.00,1.00,682.00,0.00,337.00,0.00,1092.00,151.00,176.00,75.00,14.00,2677.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,36.00,0.00,232.35,210.00,0.00,0.00,103.17,41.00,0.00,0.00,9.00
5,1914,2013-01-02,13,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Latacunga,Cotopaxi,C,15,1102.00,7.00,0.00,3.00,767.00,0.00,231.00,0.00,1000.00,111.00,195.00,48.00,28.54,2587.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,17.00,0.00,137.91,241.00,0.00,0.00,102.38,25.00,0.00,0.00,1.00
6,1947,2013-01-02,14,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Riobamba,Chimborazo,C,7,2002.00,8.00,0.00,3.00,744.00,0.00,409.00,0.00,1716.00,154.00,204.00,53.00,119.34,3257.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23.00,10.00,0.00,368.73,493.00,0.00,0.00,98.18,24.00,0.00,0.00,9.00
7,1980,2013-01-02,15,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Ibarra,Imbabura,C,15,1622.00,4.00,0.00,5.00,938.00,0.00,270.00,0.00,1300.00,152.00,219.00,73.00,33.00,3130.00,9.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,16.00,0.00,84.81,312.00,0.00,0.00,0.00,22.00,0.00,0.00,4.00
8,2013,2013-01-02,16,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Santo Domingo,Santo Domingo de los Tsachilas,C,3,1167.00,5.00,0.00,0.00,389.00,0.00,227.00,0.00,1361.00,122.00,295.00,155.00,29.00,3176.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,17.00,0.00,12.58,275.00,0.00,0.00,117.99,37.00,0.00,0.00,5.00
9,2046,2013-01-02,17,0,93.14,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,C,12,1580.00,8.00,0.00,2.00,1011.00,0.00,282.00,0.00,1329.00,185.00,378.00,139.00,48.52,3431.00,6.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.00,120.00,0.00,226.37,214.00,0.00,0.00,223.62,47.00,0.00,0.00,16.00


#### Replace nan values with zero

In [78]:
final_df.fillna(0, inplace = True)
final_df.head()

,id,date,store_nbr,onpromotion,dcoilwtico,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster,transactions,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,EGGS,FROZEN FOODS,GROCERY I,GROCERY II,HARDWARE,HOME AND KITCHEN I,HOME AND KITCHEN II,HOME APPLIANCES,HOME CARE,LADIESWEAR,LAWN AND GARDEN,LINGERIE,"LIQUOR,WINE,BEER",MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,561,2013-01-01,25,0,93.14,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00,0.00,0.00,2.00,810.00,0.00,180.59,0.00,186.00,143.00,71.09,46.00,29.65,700.00,15.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,5.00,105.00,0.00,110.80,25.00,0.00,0.00,42.64,37.85,0.00,0.00,0.00
1,1782,2013-01-02,1,0,93.14,0,0.00,0.00,0.00,0.00,0.00,Quito,Pichincha,D,13,2111.00,2.00,0.00,2.00,1091.00,0.00,470.65,0.00,1060.00,579.00,164.07,246.00,131.00,2652.00,31.00,3.00,0.00,0.00,0.00,0.00,0.00,10.00,13.00,67.00,0.00,369.10,194.00,0.00,0.00,247.30,47.00,0.00,0.00,38.03
2,1815,2013-01-02,10,0,93.14,0,0.00,0.00,0.00,0.00,0.00,Quito,Pichincha,C,15,1293.00,3.00,0.00,6.00,396.00,0.00,151.00,0.00,1110.00,101.00,276.00,57.00,41.16,2579.00,8.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,6.00,0.00,291.14,223.00,0.00,0.00,146.78,26.00,0.00,0.00,3.00
3,1848,2013-01-02,11,0,93.14,0,0.00,0.00,0.00,0.00,0.00,Cayambe,Pichincha,B,6,3547.00,12.00,0.00,7.00,3443.00,0.00,763.00,0.00,3260.00,296.00,527.29,140.00,57.77,7736.00,15.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,29.00,0.00,832.54,604.00,0.00,0.00,548.03,31.00,0.00,0.00,5.00
4,1881,2013-01-02,12,0,93.14,0,0.00,0.00,0.00,0.00,0.00,Latacunga,Cotopaxi,C,15,1362.00,4.00,0.00,1.00,682.00,0.00,337.00,0.00,1092.00,151.00,176.00,75.00,14.00,2677.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,36.00,0.00,232.35,210.00,0.00,0.00,103.17,41.00,0.00,0.00,9.00


#### Add flag column for workday

In [79]:
final_df['description'] = final_df['description'].replace(0, 'Work Day')

,id,date,store_nbr,onpromotion,dcoilwtico,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster,transactions,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,EGGS,FROZEN FOODS,GROCERY I,GROCERY II,HARDWARE,HOME AND KITCHEN I,HOME AND KITCHEN II,HOME APPLIANCES,HOME CARE,LADIESWEAR,LAWN AND GARDEN,LINGERIE,"LIQUOR,WINE,BEER",MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,561,2013-01-01,25,0,93.14,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00,0.00,0.00,2.00,810.00,0.00,180.59,0.00,186.00,143.00,71.09,46.00,29.65,700.00,15.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,5.00,105.00,0.00,110.80,25.00,0.00,0.00,42.64,37.85,0.00,0.00,0.00
1,1782,2013-01-02,1,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Quito,Pichincha,D,13,2111.00,2.00,0.00,2.00,1091.00,0.00,470.65,0.00,1060.00,579.00,164.07,246.00,131.00,2652.00,31.00,3.00,0.00,0.00,0.00,0.00,0.00,10.00,13.00,67.00,0.00,369.10,194.00,0.00,0.00,247.30,47.00,0.00,0.00,38.03
2,1815,2013-01-02,10,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Quito,Pichincha,C,15,1293.00,3.00,0.00,6.00,396.00,0.00,151.00,0.00,1110.00,101.00,276.00,57.00,41.16,2579.00,8.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,6.00,0.00,291.14,223.00,0.00,0.00,146.78,26.00,0.00,0.00,3.00
3,1848,2013-01-02,11,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Cayambe,Pichincha,B,6,3547.00,12.00,0.00,7.00,3443.00,0.00,763.00,0.00,3260.00,296.00,527.29,140.00,57.77,7736.00,15.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,29.00,0.00,832.54,604.00,0.00,0.00,548.03,31.00,0.00,0.00,5.00
4,1881,2013-01-02,12,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Latacunga,Cotopaxi,C,15,1362.00,4.00,0.00,1.00,682.00,0.00,337.00,0.00,1092.00,151.00,176.00,75.00,14.00,2677.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,36.00,0.00,232.35,210.00,0.00,0.00,103.17,41.00,0.00,0.00,9.00


In [80]:
final_df['is_workday'] = final_df['description'].apply(lambda x: 1 if 'Work Day' in x else 0)

,id,date,store_nbr,onpromotion,dcoilwtico,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster,transactions,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,EGGS,FROZEN FOODS,GROCERY I,GROCERY II,HARDWARE,HOME AND KITCHEN I,HOME AND KITCHEN II,HOME APPLIANCES,HOME CARE,LADIESWEAR,LAWN AND GARDEN,LINGERIE,"LIQUOR,WINE,BEER",MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD,is_workday
0,561,2013-01-01,25,0,93.14,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00,0.00,0.00,2.00,810.00,0.00,180.59,0.00,186.00,143.00,71.09,46.00,29.65,700.00,15.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,5.00,105.00,0.00,110.80,25.00,0.00,0.00,42.64,37.85,0.00,0.00,0.00,0
1,1782,2013-01-02,1,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Quito,Pichincha,D,13,2111.00,2.00,0.00,2.00,1091.00,0.00,470.65,0.00,1060.00,579.00,164.07,246.00,131.00,2652.00,31.00,3.00,0.00,0.00,0.00,0.00,0.00,10.00,13.00,67.00,0.00,369.10,194.00,0.00,0.00,247.30,47.00,0.00,0.00,38.03,1
2,1815,2013-01-02,10,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Quito,Pichincha,C,15,1293.00,3.00,0.00,6.00,396.00,0.00,151.00,0.00,1110.00,101.00,276.00,57.00,41.16,2579.00,8.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,6.00,0.00,291.14,223.00,0.00,0.00,146.78,26.00,0.00,0.00,3.00,1
3,1848,2013-01-02,11,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Cayambe,Pichincha,B,6,3547.00,12.00,0.00,7.00,3443.00,0.00,763.00,0.00,3260.00,296.00,527.29,140.00,57.77,7736.00,15.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,29.00,0.00,832.54,604.00,0.00,0.00,548.03,31.00,0.00,0.00,5.00,1
4,1881,2013-01-02,12,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Latacunga,Cotopaxi,C,15,1362.00,4.00,0.00,1.00,682.00,0.00,337.00,0.00,1092.00,151.00,176.00,75.00,14.00,2677.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,36.00,0.00,232.35,210.00,0.00,0.00,103.17,41.00,0.00,0.00,9.00,1


#### Calender information
1. Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.

2. A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [81]:
# Add public payday flag column on the 15th and the last day of every month and a flag column for the three weeks after earthquake on april 16 2016
final_df['public_payday_flag'] = final_df['date'].apply(
    lambda x: 1 if (x.day == 15 or x.day == x.days_in_month) else 0)

earthquake_date = pd.Timestamp('2016-04-16')
earthquake_end_date = earthquake_date + pd.Timedelta(weeks=3)

# This flag is 1 for dates within three weeks after the earthquake
final_df['earthquake_aftermath_flag'] = final_df['date'].apply(
    lambda x: 1 if earthquake_date <= x <= earthquake_end_date else 0)

final_df.head()

,id,date,store_nbr,onpromotion,dcoilwtico,description,transferred,bridge,holiday_national,holiday_local,holiday_regional,city,state,type_y,cluster,transactions,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,EGGS,FROZEN FOODS,GROCERY I,GROCERY II,HARDWARE,HOME AND KITCHEN I,HOME AND KITCHEN II,HOME APPLIANCES,HOME CARE,LADIESWEAR,LAWN AND GARDEN,LINGERIE,"LIQUOR,WINE,BEER",MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD,is_workday,public_payday_flag,earthquake_aftermath_flag
0,561,2013-01-01,25,0,93.14,Primer dia del ano,0.00,0.00,1.00,0.00,0.00,Salinas,Santa Elena,D,1,770.00,0.00,0.00,2.00,810.00,0.00,180.59,0.00,186.00,143.00,71.09,46.00,29.65,700.00,15.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,5.00,105.00,0.00,110.80,25.00,0.00,0.00,42.64,37.85,0.00,0.00,0.00,0,0,0
1,1782,2013-01-02,1,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Quito,Pichincha,D,13,2111.00,2.00,0.00,2.00,1091.00,0.00,470.65,0.00,1060.00,579.00,164.07,246.00,131.00,2652.00,31.00,3.00,0.00,0.00,0.00,0.00,0.00,10.00,13.00,67.00,0.00,369.10,194.00,0.00,0.00,247.30,47.00,0.00,0.00,38.03,1,0,0
2,1815,2013-01-02,10,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Quito,Pichincha,C,15,1293.00,3.00,0.00,6.00,396.00,0.00,151.00,0.00,1110.00,101.00,276.00,57.00,41.16,2579.00,8.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,6.00,0.00,291.14,223.00,0.00,0.00,146.78,26.00,0.00,0.00,3.00,1,0,0
3,1848,2013-01-02,11,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Cayambe,Pichincha,B,6,3547.00,12.00,0.00,7.00,3443.00,0.00,763.00,0.00,3260.00,296.00,527.29,140.00,57.77,7736.00,15.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,29.00,0.00,832.54,604.00,0.00,0.00,548.03,31.00,0.00,0.00,5.00,1,0,0
4,1881,2013-01-02,12,0,93.14,Work Day,0.00,0.00,0.00,0.00,0.00,Latacunga,Cotopaxi,C,15,1362.00,4.00,0.00,1.00,682.00,0.00,337.00,0.00,1092.00,151.00,176.00,75.00,14.00,2677.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,36.00,0.00,232.35,210.00,0.00,0.00,103.17,41.00,0.00,0.00,9.00,1,0,0


### Datatypes conversion for memory saving

In [82]:
final_df['description'] = final_df['description'].astype('category')
final_df['city'] = final_df['city'].astype('category')
final_df['state'] = final_df['state'].astype('category')
final_df['type_y'] = final_df['type_y'].astype('category')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84207 entries, 0 to 84206
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          84207 non-null  int64         
 1   date                        84207 non-null  datetime64[ns]
 2   store_nbr                   84207 non-null  int64         
 3   onpromotion                 84207 non-null  int64         
 4   dcoilwtico                  84207 non-null  float64       
 5   description                 84207 non-null  category      
 6   transferred                 84207 non-null  float64       
 7   bridge                      84207 non-null  float64       
 8   holiday_national            84207 non-null  float64       
 9   holiday_local               84207 non-null  float64       
 10  holiday_regional            84207 non-null  float64       
 11  city                        84207 non-null  category  

In [83]:
# final_df.to_parquet('store_train_complete.parquet', compression='snappy', engine='pyarrow')